In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

from PIL import Image, ImageFont, ImageDraw
from io import BytesIO
import matplotlib.pyplot as plt

import textwrap
from selenium.webdriver.chrome.options import Options


In [2]:
def break_fix(text, width, font, draw):
    if not text:
        return
    lo = 0
    hi = len(text)
    while lo < hi:
        mid = (lo + hi + 1) // 2
        t = text[:mid]
        w, h = draw.textsize(t, font=font)
        if w <= width:
            lo = mid
        else:
            hi = mid - 1
    t = text[:lo]
    w, h = draw.textsize(t, font=font)
    yield t, w, h
    yield from break_fix(text[lo:], width, font, draw)

def fit_text(img, text, color, font):
    width = img.size[0] - 2
    draw = ImageDraw.Draw(img)
    pieces = list(break_fix(text, width, font, draw))
    height = sum(p[2] for p in pieces)
    if height > img.size[1]:
        raise ValueError("text doesn't fit")
    y = (img.size[1] - height) // 2
    for t, w, h in pieces:
        x = (img.size[0] - w) // 2
        draw.text((x, y), t, font=font, fill=color)
        y += h

In [4]:

driver = webdriver.Chrome()
 
driver.maximize_window()
url = "https://www.randomtextgenerator.com/"
driver.get(url)

font=ImageFont.truetype('/Library/Fonts/Arial Unicode.ttf', 16)
languages = ['pt', 'ru', 'sr', 'es']

for lang in languages:
    for i in range(5000):
        path = 'language_data/' + lang + '/' + str(i) + '.png'
        driver.refresh()

        newsletter = driver.find_element(By.XPATH, value=f"//option[@value='{lang}']")
        newsletter.click()

        driver.find_element(By.XPATH, "//input[@type='submit']").click()

        text = driver.find_element(By.ID, "randomtext_box").text
        text = text.split('\n')[0]
        im = Image.new("RGB", (512, 512), "white")
        fit_text(im, text, (0,0,0), font)

        im.save(path)
    
driver.quit()

